In [1]:
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import sys
import itertools
from collections import namedtuple
from pathlib import Path

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 1
%aimport analyze

with open('plots/style.json') as f:
    mpl.rcParams.update(json.load(f))

## Wildstyle failures

In [2]:
ws = analyze.Analysis('results/wildstyle')
print(ws)

Analysis for Celeritas v0.2.0 on wildstyle


In [3]:
ftab = analyze.make_failure_table(ws.failures())
ftab.to_frame()

,Failure
cms2018+field+msc/vecgeom+cpu (0),internal assertion failed: `succeeded` at `Fie...
cms2018+field+msc/vecgeom+cpu (1),internal assertion failed: `succeeded` at `Fie...
cms2018+field+msc/vecgeom+gpu (0),`celeritas: internal assertion failed: succeeded`
cms2018+field+msc/vecgeom+gpu (1),`celeritas: internal assertion failed: succeeded`
cms2018/vecgeom+cpu (0),internal assertion failed: `speed > 0` at `Alo...
cms2018/vecgeom+cpu (1),internal assertion failed: `speed > 0` at `Alo...
cms2018/vecgeom+gpu (0),`celeritas: internal assertion failed: speed > 0`
cms2018/vecgeom+gpu (1),`celeritas: internal assertion failed: speed > 0`
testem3-flat+field/orange+cpu (0),internal assertion failed: `speed > 0` at `Alo...
testem3-flat+field/orange+cpu (1),internal assertion failed: `speed > 0` at `Alo...


## Summit results

In [4]:
summit = analyze.Analysis('results/summit')
print(summit)
some_results = summit.load_results(('testem3-flat','orange','gpu'), 0)

Analysis for Celeritas v0.2.0 on summit


In [5]:
failures = summit.failures()

In [6]:
analyze.make_failure_table(failures).to_frame()

,Failure
cms2018+field+msc/vecgeom+cpu (0),runtime error: `insufficient capacity (1048576...
cms2018+field+msc/vecgeom+cpu (1),runtime error: `insufficient capacity (1048576...
cms2018+field+msc/vecgeom+cpu (2),runtime error: `insufficient capacity (1048576...
cms2018+field+msc/vecgeom+cpu (3),runtime error: `insufficient capacity (1048576...
cms2018+field+msc/vecgeom+cpu (4),(unknown failure)
cms2018+field+msc/vecgeom+cpu (5),runtime error: `insufficient capacity (1048576...
simple-cms+field+msc/orange+cpu (0),`/ccs/home/s3j/.local/src/celeritas-summit/app...
simple-cms+field+msc/orange+cpu (1),`/ccs/home/s3j/.local/src/celeritas-summit/app...
simple-cms+field+msc/orange+cpu (2),`/ccs/home/s3j/.local/src/celeritas-summit/app...
simple-cms+field+msc/orange+cpu (3),`/ccs/home/s3j/.local/src/celeritas-summit/app...


In [7]:
failures.groupby(['problem', 'geo', 'arch']).count().unstack().fillna(0)

failure                      condition      file      line      type       \
arch                               cpu  gpu  cpu  gpu  cpu  gpu  cpu  gpu   
problem              geo                                                    
cms2018+field+msc    vecgeom       5.0  0.0  5.0  0.0  5.0  0.0  5.0  0.0   
simple-cms+field     orange        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
simple-cms+field+msc orange        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
                     vecgeom       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
testem3-flat+field   orange        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
testem3-flat+msc     orange        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

failure                      what      which      stderr      stdout       
arch                          cpu  gpu   cpu  gpu    cpu  gpu    cpu  gpu  
problem              geo                                                   
cms2018+field+msc    vecgeom  5.0  0.0   5.0  0.0    1.0  0.0    1.0  0.0  
simple-cms+field     orange   0.0  0.0   0.0  0.0    6.0  0.0    0.0  0.0  
simple-cms+field+msc orange   0.0  0.0   0.0  0.0    6.0  0.0    0.0  0.0  
                     vecgeom  0.0  0.0   0.0  0.0    6.0  0.0    0.0  0.0  
testem3-flat+field   orange   0.0  0.0   0.0  0.0    3.0  3.0    3.0  3.0  
testem3-flat+msc     orange   0.0  0.0   0.0  0.0    5.0  6.0    5.0  6.0

In [24]:
summed = analyze.summarize_instances(summit.result[[
    'avg_steps_per_primary',
    'avg_time_per_primary',
    'avg_time_per_step',
    'num_steps',
    'total_time',
    'unconverged',
    'num_primaries',
    'num_events',
    'slot_occupancy']])
#summed.xs('mean', axis=1, level=1).to_csv('summit.csv')

In [9]:
def float_fmt_transform(digits):
    format = "{{:.{}f}}".format(digits).format
    def transform(val):
        if np.isnan(val):
            return "---"
        return format(val)
    return transform

In [10]:
times = summed[('total_time', 'mean')].unstack()
times.style.format(float_fmt_transform(2))

In [11]:
unconv = summed[('unconverged', 'mean')]
unconv[unconv > 0].unstack('arch') 

arch                          cpu    gpu
problem              geo                
simple-cms+field     orange   1.0   87.0
simple-cms+field+msc orange   2.0  339.0
                     vecgeom  2.0  339.0

In [12]:
problems = summit.problems()
problem_to_abbr = summit.problem_to_abbr(problems)
p_to_i = dict(zip(problems, itertools.count()))

In [13]:
speedup = analyze.get_cpugpu_ratio(summed['total_time'])
fig, ax = plt.subplots()
summit.plot_results(ax, speedup)
ax.set_ylabel("Speedup (7-CPU / 1-GPU wall time)")
ax.set_ylim([0, None])
analyze.annotate_metadata(ax, summit);
fig.savefig('plots/speedups.pdf', transparent=True)
plt.close()

In [14]:
fig, axes = plt.subplots(nrows=2, figsize=(4,4), subplot_kw=dict(yscale='log'))
for (ax, q) in zip(axes, ['step', 'primary']):
    summit.plot_results(ax, analyze.inverse_summary(summed['avg_time_per_' + q]))
    ax.set_ylabel(q + ' per sec')
    ax.legend()
fig.savefig('plots/steps-vs-primaries.png', dpi=300)
plt.close()

In [20]:
summed

avg_steps_per_primary                 \
                                                  count           mean   
problem              geo     arch                                        
cms2018              vecgeom cpu                    6.0   82243.863566   
                             gpu                    6.0   37050.256557   
cms2018+field+msc    vecgeom cpu                    0.0            NaN   
simple-cms+field     orange  cpu                    6.0     530.204875   
                             gpu                    6.0     106.885110   
simple-cms+field+msc orange  cpu                    6.0     416.872640   
                             gpu                    6.0      79.092143   
                     vecgeom cpu                    6.0     416.872640   
                             gpu                    6.0      79.092143   
simple-cms+msc       orange  cpu                    6.0   64765.444377   
                             gpu                    6.0   29147.921245   
testem15             orange  cpu                    6.0   54926.513143   
                             gpu                    6.0   24721.853535   
testem15+field       orange  cpu                    6.0   54926.513143   
                             gpu                    6.0   24721.853535   
testem15+field+msc   orange  cpu                    6.0   54924.890666   
                             gpu                    6.0   24726.364908   
                     vecgeom cpu                    6.0   54924.890666   
                             gpu                    6.0   24726.364908   
testem3-flat         orange  cpu                    6.0  120108.541585   
                             gpu                    6.0   54059.576886   
                     vecgeom cpu                    6.0  120108.541585   
                             gpu                    6.0   54059.576886   
testem3-flat+field   orange  cpu                    3.0  118464.623291   
                             gpu                    3.0   53311.073736   
testem3-flat+msc     orange  cpu                    1.0  142269.325684   
                             gpu                    0.0            NaN   

                                             avg_time_per_primary            \
                                         std                count      mean   
problem              geo     arch                                             
cms2018              vecgeom cpu   71.725247                  6.0  0.024271   
                             gpu   19.888253                  6.0  0.001185   
cms2018+field+msc    vecgeom cpu         NaN                  0.0       NaN   
simple-cms+field     orange  cpu    0.000240                  6.0  0.646609   
                             gpu    0.000060                  6.0  0.004391   
simple-cms+field+msc orange  cpu    0.000199                  6.0  0.330557   
                             gpu    0.000060                  6.0  0.001164   
                     vecgeom cpu    0.000199                  6.0  0.330686   
                             gpu    0.000060                  6.0  0.001167   
simple-cms+msc       orange  cpu   26.352302                  6.0  0.017459   
                             gpu   12.221082                  6.0  0.000345   
testem15             orange  cpu    7.541768                  6.0  0.011669   
                             gpu    4.936900                  6.0  0.000268   
testem15+field       orange  cpu    7.541768                  6.0  0.014997   
                             gpu    4.936900                  6.0  0.000263   
testem15+field+msc   orange  cpu    8.496707                  6.0  0.018680   
                             gpu    3.628817                  6.0  0.000304   
                     vecgeom cpu    8.496707                  6.0  0.018200   
                             gpu    3.628817                  6.0  0.000296   
testem3-flat         orange  cpu   23.875669                  6.0  0.021448

In [25]:
event_rate = analyze.calc_event_rate(summit, summed)

In [26]:
event_rate.xs('testem3-flat', level='problem')

count      mean       std
geo     arch                           
orange  cpu     6.0  0.035865  0.000270
        gpu     6.0  2.066271  0.168098
vecgeom cpu     6.0  0.037569  0.000378
        gpu     6.0  1.884863  0.121249

In [27]:
(fig, (time_ax, occ_ax)) = plt.subplots(
    nrows=2, figsize=(4, 4),
    gridspec_kw=dict(height_ratios=[3, 1])
)
time_ax.set_yscale('log')
summit.plot_results(time_ax, event_rate)
time_ax.set_ylabel(r"Event rate [1/s]")
time_ax.legend()
time_ax.set_xticklabels([])
summit.plot_results(occ_ax, summed['slot_occupancy'])
occ_ax.set_ylabel("Slot occupancy")
analyze.annotate_metadata(ax, summit)
fig.savefig('plots/rate-occupancy.pdf', transparent=True)
plt.close()

In [29]:
speedup.dropna().style.format("{:.1f}".format)

## Action fraction pie charts


In [30]:
mean_action_times = summit.action_times().xs('mean', axis=1, level=1).T
mean_action_times.sort_index(inplace=True)

ValueError: Cannot describe a DataFrame without columns

In [ ]:
for ext in ["", "+field+msc"]:
    prob_geo = ('cms2018'+ext, 'vecgeom')
    temp = mean_action_times.xs(prob_geo, axis=1, level=('problem', 'geo')).dropna()

    for (arch, series) in temp.items():
        (fig, ax) = plt.subplots(figsize=(4, 4))
        ax.pie(series, labels=series.index, autopct='%1.1f%%', pctdistance=0.85)
        ax.axis('equal')
        name = prob_geo + (arch,)
        slashname = "/".join(name)
        fig.text(
            0.98, 0.02, f"{slashname}\n{summit.version} on {summit.system}",
            va='bottom', ha='right',
            fontstyle='italic', color=(0.5,)*3, size='xx-small',
            zorder=-100
        )
        dashname = "-".join(name)
        fig.savefig(f'plots/{dashname}.pdf', transparent=True)
        plt.close()

### Plot per-step timing on GPU

In [31]:
cms = [summit.load_results((p, 'vecgeom', 'gpu'), 0)
       for p in ['cms2018', 'cms2018+field+msc']]

for plot, label in [(analyze.plot_counts, 'counts'),
                    (analyze.plot_accum_time, 'time')]:
    (fig, axes) = plt.subplots(ncols=2, figsize=(8, 2))
    
    for (i, ax, data) in zip(itertools.count(), axes, cms):
        objs = plot(ax, data)
        analyze.annotate_metadata(ax, data['_metadata'])
        if i == 0:
            objs['oax'].set_ylabel(None)
        elif i == 1:
            objs['ax'].set_ylabel(None)
    fig.savefig(f'plots/cms-{label}.pdf', transparent=True)
    plt.close()

FileNotFoundError: [Errno 2] No such file or directory: 'results/summit/cms2018+field+msc-vecgeom-gpu/0.json'

## Crusher

In [32]:
crusher = analyze.Analysis('results/crusher')
print(crusher)

Analysis for Celeritas v0.2.0-1+49ccc7c8 on crusher


In [33]:
# VecGeom failures aren't really failures; just missing capability
failures = crusher.failures().xs('orange', level='geo').fillna(1)
failures.groupby(['problem', 'arch']).count().unstack()

failure              stderr      stdout     
arch                    cpu  gpu    cpu  gpu
problem                                     
simple-cms+field        8.0  NaN    8.0  NaN
simple-cms+field+msc    8.0  NaN    8.0  NaN
testem3-flat+field      4.0  4.0    4.0  4.0
testem3-flat+msc        7.0  8.0    7.0  8.0

In [34]:
failures['stderr'].fillna(1).unstack('instance').dropna(axis=0)

,instance,0,1,2,3,4,5,6,7
problem,arch,,,,,,,,
simple-cms+field+msc,cpu,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...
simple-cms+field,cpu,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...
testem3-flat+msc,gpu,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...,[/ccs/home/s3j/.local/src/celeritas-crusher/ap...


In [35]:
csum = analyze.summarize_instances(crusher.result[~crusher.invalid][[
    'avg_steps_per_primary',
    'avg_time_per_primary',
    'avg_time_per_step',
    'num_steps',
    'total_time',
    'unconverged',
    'slot_occupancy',
    'num_primaries',
    'num_events',
]])

In [36]:
csum[('total_time', 'mean')].unstack()

,arch,cpu,gpu
problem,geo,,
simple-cms+field,orange,NaN,43.741347
simple-cms+field+msc,orange,NaN,11.267921
simple-cms+msc,orange,83.134969,2.175320
testem15,orange,89.462248,1.660482
testem15+field,orange,59.999452,1.850357
testem15+field+msc,orange,89.137889,2.165253
testem3-flat,orange,107.263641,3.244948
testem3-flat+field,orange,140.217632,6.640344
testem3-flat+msc,orange,174.890197,NaN


In [37]:
rel_err = csum.xs('std', axis=1, level=1) / csum.xs('mean', axis=1, level=1)
high_err = rel_err > 0.02
rel_err[high_err].dropna(how='all').dropna(how='all', axis=1)

avg_time_per_primary  avg_time_per_step  \
problem            geo    arch                                            
simple-cms+msc     orange cpu               0.172376           0.172437   
                          gpu                    NaN                NaN   
testem15           orange cpu               0.422068           0.422085   
                          gpu                    NaN                NaN   
testem15+field     orange cpu               0.321016           0.321043   
                          gpu                    NaN                NaN   
testem15+field+msc orange cpu               0.164774           0.164771   
                          gpu                    NaN                NaN   
testem3-flat       orange cpu               0.539319           0.539219   
testem3-flat+field orange cpu               0.348533           0.348523   
                          gpu                    NaN                NaN   

                                total_time  slot_occupancy  
problem            geo    arch                              
simple-cms+msc     orange cpu     0.172376             NaN  
                          gpu          NaN        0.035145  
testem15           orange cpu     0.422068             NaN  
                          gpu          NaN        0.026928  
testem15+field     orange cpu     0.321016             NaN  
                          gpu          NaN        0.026928  
testem15+field+msc orange cpu     0.164774             NaN  
                          gpu          NaN        0.024806  
testem3-flat       orange cpu     0.539319             NaN  
testem3-flat+field orange cpu     0.348533             NaN  
                          gpu          NaN        0.031954

In [38]:
analyze.get_cpugpu_ratio(csum['total_time'])

,,mean,std
problem,geo,,
simple-cms+field,orange,NaN,NaN
simple-cms+field+msc,orange,NaN,NaN
simple-cms+msc,orange,38.217353,6.601693
testem15,orange,53.877265,22.741743
testem15+field,orange,32.425875,10.413711
testem15+field+msc,orange,41.167420,6.786948
testem3-flat,orange,33.055577,17.828039
testem3-flat+field,orange,21.116020,7.360462
testem3-flat+msc,orange,NaN,NaN


In [39]:
crusher_times = csum['total_time']
crusher_times

count        mean        std
problem              geo    arch                              
simple-cms+field     orange gpu     8.0   43.741347   0.250860
simple-cms+field+msc orange gpu     8.0   11.267921   0.115858
simple-cms+msc       orange cpu     8.0   83.134969  14.330453
                            gpu     8.0    2.175320   0.024413
testem15             orange cpu     8.0   89.462248  37.759161
                            gpu     8.0    1.660482   0.008983
testem15+field       orange cpu     8.0   59.999452  19.260794
                            gpu     8.0    1.850357   0.017432
testem15+field+msc   orange cpu     8.0   89.137889  14.687576
                            gpu     8.0    2.165253   0.011692
testem3-flat         orange cpu     8.0  107.263641  57.849365
                            gpu     8.0    3.244948   0.013421
testem3-flat+field   orange cpu     4.0  140.217632  48.870489
                            gpu     4.0    6.640344   0.034761
testem3-flat+msc     orange cpu     1.0  174.890197        NaN

In [43]:
crusher_rates = analyze.calc_event_rate(crusher, csum)
summit_rates = analyze.calc_event_rate(summit, summed.loc[crusher_times.index])

counts = {
    ('summit', 'cpu'): 7,
    ('summit', 'gpu'): 1,
    ('crusher', 'cpu'): 8,
    ('crusher', 'gpu'): 1,
}

In [44]:
(crusher_rates['mean'] / summit_rates['mean']).unstack()

,arch,cpu,gpu
problem,geo,,
simple-cms+field,orange,NaN,0.913599
simple-cms+field+msc,orange,NaN,0.940144
simple-cms+msc,orange,0.860179,1.444362
testem15,orange,0.534269,1.471178
testem15+field,orange,1.023794,1.291316
testem15+field+msc,orange,0.858386,1.278250
testem3-flat,orange,0.819027,1.044006
testem3-flat+field,orange,1.073450,0.859865
testem3-flat+msc,orange,0.921367,NaN


In [45]:
fig, ax = plt.subplots()
ax.set_yscale('log')
for offset, color, machine, rates in [(-0.05, '#7A954F', 'Summit', summit_rates),
                                      (0.05, '#BC5544', 'Crusher', crusher_rates)]:
    for arch in ['cpu', 'gpu']:
        summary = rates.xs(arch, level='arch')
        index = np.array([p_to_i[p]
                          for p in summary.index.get_level_values('problem')], dtype=float)
        index += offset
    
        mark = analyze.ARCH_SHAPES[arch]
        count = counts[(machine.lower(), arch)]
        arch = arch.upper()
        ax.errorbar(index, summary['mean'], summary['std'],
                    capsize=0, fmt='none', ecolor=(0.2,)*3)
        scat = ax.scatter(index, summary['mean'], c=color, marker=mark,
                         label=f"{machine} ({count} {arch})")    
xax = ax.get_xaxis()
xax.set_ticks(np.arange(len(problems)))
xax.set_ticklabels(list(problem_to_abbr.values()), rotation=90)
grid = ax.grid()
ax.set_axisbelow(True)
ax.legend()
ax.set_ylabel(r"Event rate [1/s]")
analyze.annotate_metadata(ax, summit)
fig.savefig('plots/crusher-vs-summit.pdf')
plt.close()